〈入力データの説明〉
  
  感情の指標のリスト
  [C1, C2, C3, C4, C5, C6]

 ・ 印象クラス

  C1:堂々とした印象/
  C2:元気が出る/
  C3:切ない/
  C4:激しい/
  C5:滑稽/
  C6:可愛い


<今回>

InputData = [3.0000,1.7500,2.2500,2.2500,2.0000,1.7500]

In [174]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline

import requests
from bs4 import BeautifulSoup
import pickle

import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [175]:
NumberOfClass = 3        #クラスタリング時のクラス数
TimesOfLearning = 3000   #学習回数

In [176]:
#データセットを読み込む
ds1 = pd.read_csv('NicoNicoVideoMoodDataset500ChorusMovie.tsv',delimiter='\t', header=None,names=['target', 'C1','C2','C3','C4','C5','C6','Valence','Arousal'])
ds2 = pd.read_csv('NicoNicoVideoMoodDataset500ChorusAudio.tsv',delimiter='\t', header=None,names=['target', 'C1','C2','C3','C4','C5','C6','Valence','Arousal'])
ds3 = pd.read_csv('NicoNicoVideoMoodDataset500ChorusVisual.tsv',delimiter='\t', header=None,names=['target', 'C1','C2','C3','C4','C5','C6','Valence','Arousal'])

ds = pd.concat([ds1, ds2, ds3]) #3つのdsを結合
ds = ds.reset_index(drop=True)  #インデックスの振り直し
ds.to_csv("train_ds.csv")

In [177]:
#dtのC1～C6までの列を取得
X = ds[['C1','C2','C3','C4','C5','C6',]]

In [178]:
# (NumberOfClass)個のクラスに分ける
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=NumberOfClass, max_iter=30, init="random")
cluster = kmeans.fit_predict(X.values)

In [179]:
X["cluster"] = [x for x in cluster]
X.to_csv("train_X.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [180]:
ds = pd.read_csv("train_X.csv",index_col=0)
y = X.loc[:,'cluster']
X = ds.iloc[:,1:]

In [181]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0)

In [182]:
# データを格納する
# 学習用
lgb_train = lgb.Dataset(X_train, y_train,free_raw_data=False)
# 検証用
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)

In [183]:
# パラメータを設定
params = {'task': 'train',                # 学習、トレーニング ⇔　予測predict
          'boosting_type': 'gbdt',        # 勾配ブースティング
          'objective': 'multiclass',      # 目的関数：多値分類、マルチクラス分類
          'metric': 'multi_logloss',      # 分類モデルの性能を測る指標
          'num_class': NumberOfClass,                 # 目的変数のクラス数
          'learning_rate': 0.02,          # 学習率（初期値0.1）
          'min_data_in_leaf': 1,          # データの最小数（初期値20）
         }

In [ ]:
# 学習
evaluation_results = {}                                     # 学習の経過を保存する箱
model = lgb.train(params,                                   # 上記で設定したパラメータ
                  lgb_train,                                # 使用するデータセット
                  num_boost_round=TimesOfLearning,          # 学習の回数
                  valid_names=['train', 'valid'],           # 学習経過で表示する名称
                  valid_sets=[lgb_train, lgb_eval],         # モデル検証のデータセット
                  evals_result=evaluation_results,          # 学習の経過を保存
                  verbose_eval=10)                          # 学習の経過の表示(10回毎)

# 最もスコアが良いときのラウンドを保存
optimum_boost_rounds = model.best_iteration

#学習モデルを保存
pickle.dump(model, open("Model.pkl", 'wb'))

In [188]:
# Accuracy の計算
model = pickle.load(open('Model.pkl', 'rb'))
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)
accuracy = sum(y_test == y_pred_max) / len(y_test)
print('accuracy:', accuracy)

accuracy: 1.0


In [189]:
def main(Emotion):
  ds = pd.read_csv("train_ds.csv",index_col=0)
  model = pickle.load(open('Model.pkl', 'rb'))
  E = pd.Series(Emotion, index = ["C1","C2","C3","C4","C5","C6"])  #受けとったリストをデータフレームに変換
  y_pred = model.predict(E, num_iteration=model.best_iteration)    #予測
  Pre = y_pred.argmax()                                            #予測結果から最も高い確率のものを選出
  Apply = y[y==Pre]                                                #予測結果と同クラスの曲IDを抽出
  ID = ds["target"]                                                #曲IDのみのデータフレーム生成
  
  list = []
  for i in Apply.index:                                            #インデックス取得
    list.append(i)

  Recommended_Index = random.sample(list, 3)                       #同クラスの曲の中からランダムに３曲選出
  print("\nWe recommend following musics\n")

  Recommended_ID = []
  for j in Recommended_Index:
    Recommended_ID.append(ID[j])
  
  for bottom in Recommended_ID:
    Recommended_url = f"https://www.nicovideo.jp/watch/{bottom}"
    html = requests.get(Recommended_url)
    soup = BeautifulSoup(html.content, "html.parser")

    Recommended_name = soup.find("title").text.replace("- ニコニコ動画", "")

    Recommend_sentence = f"{Recommended_name} : {Recommended_url}"
    print(Recommend_sentence)

In [191]:
emotion = [3.0000,1.7500,2.2500,2.2500,2.0000,1.7500] #入力データ

main(emotion)



We recommend following musics

【鏡音リン】狂騒ノ現【オリジナル/ワンオポ】  : https://www.nicovideo.jp/watch/sm11146117
【GUMI】ずっと光る砂【PV付オリジナル曲】  : https://www.nicovideo.jp/watch/sm15800552
【初音ミク】ドミノ倒シ【オリジナル曲PV付】  : https://www.nicovideo.jp/watch/sm13306896
